# Extract Sample Metadata

This notebook extracts key metadata columns (Sample ID, Location, Collection Date) from the full sample metadata table.

- For US samples: parses state from strain name (e.g., `MVs/Virginia.USA/...`)
- For Canadian samples: parses province from strain name (e.g., `MVs/Ontario.CAN/...`)

In [ ]:
import gxy
import pandas as pd
import re

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded")

In [ ]:
# Dataset ID - change this to match your metadata file
METADATA_DATASET = 13674

In [ ]:
# Download the metadata file
path = await gxy.get(METADATA_DATASET)
print(f"Downloaded: {path}")

In [ ]:
# Load the full metadata
df = pd.read_csv(path, sep='\t')
print(f"Loaded {len(df)} samples with {len(df.columns)} columns")

In [ ]:
def extract_location(row):
    """
    Extract location from sample metadata.
    
    - USA samples: parse state from strain (MVs/Virginia.USA/... -> USA:Virginia)
    - Canada samples: parse province from strain (MVs/Ontario.CAN/... -> Canada:Ontario)
    - Other countries: use country field as-is
    """
    country = str(row.get('country', '')).strip()
    strain = str(row.get('strain', ''))
    
    # USA: parse state
    if country == 'USA' and strain:
        match = re.search(r'MVs/([A-Za-z ]+)\.USA/', strain)
        if match:
            state = match.group(1).strip()
            return f"USA:{state}"
    
    # Canada: parse province
    if country == 'Canada' and strain:
        match = re.search(r'MVs/([A-Za-z ]+)\.CAN/', strain)
        if match:
            province = match.group(1).strip()
            return f"Canada:{province}"
    
    return country

# Test on a few rows
print("Testing location extraction:")
test_indices = [0, 5, 45, 55, 103]  # USA, USA, Canada Alberta, Canada BC, Romania
for idx in test_indices:
    if idx < len(df):
        loc = extract_location(df.iloc[idx])
        print(f"  {df.iloc[idx]['Sample']}: {loc}")

In [ ]:
# Extract the 3 columns we need, with enhanced location
result = pd.DataFrame({
    'sample_id': df['Sample'],
    'location': df.apply(extract_location, axis=1),
    'collection_date': df['collection_date']
})

print(f"Extracted {len(result)} rows")
result.head(10)

In [ ]:
# Filter out samples missing location or date
before = len(result)
result = result.dropna(subset=['location', 'collection_date'])
result = result[result['location'].str.strip() != '']
result = result[result['collection_date'].str.strip() != '']
after = len(result)

print(f"Filtered: {before} -> {after} samples")
print(f"Removed {before - after} samples with missing data")

In [ ]:
# Summary
print("=== SUMMARY ===")
print(f"Total samples: {len(result)}")
print(f"\nLocations:")
print(result['location'].value_counts().to_string())
print(f"\nDate range: {result['collection_date'].min()} to {result['collection_date'].max()}")

In [ ]:
# View final table
result

In [ ]:
# Save to Galaxy
result.to_csv('sample_metadata_clean.tsv', sep='\t', index=False)
await gxy.put('sample_metadata_clean.tsv', output='Sample Metadata (Clean)', ext='tabular')
print("Saved to Galaxy history!")